In [1]:
import re
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

In [2]:
CHROMEDRIVER = "../../webdrivers/chromedriver"
SEARCH_FIELD_ID = "ctl00_cphContent_txtEntityName"
HOME_URL = "https://coraweb.sos.la.gov/CommercialSearch/CommercialSearch.aspx"
SEARCH_RESULTS_ID = "ctl00_cphContent_grdSearchResults_EntityNameOrCharterNumber"
TEST_BUSINESSES = ["Whirlwind space ac llc","T & T Real Estate Dev LLC","R2h Investments LLC"]

In [3]:
service = webdriver.chrome.service.Service(CHROMEDRIVER)  
service.start()

chrome_options = Options()  
chrome_options.add_argument("--headless")  

driver = webdriver.Remote(service.service_url, desired_capabilities=chrome_options.to_capabilities())

In [4]:
def search(business_name):
    driver.get(HOME_URL)

    search_field = driver.find_element_by_id(SEARCH_FIELD_ID)
    search_field.clear()  
    search_field.send_keys(business_name)  
    search_field.send_keys(Keys.RETURN)

In [5]:
def is_on_business_page(page):
    soup = BeautifulSoup(page, 'lxml')
    if soup.find(id=SEARCH_RESULTS_ID) is None:
#         print('on registered page')
        return True
    
#     print('on search page')
    return False

In [6]:
def get_business_details(business_name,business_page):
    soup = BeautifulSoup(business_page, "lxml")
    registered_agents = soup.find(id="ctl00_cphContent_pnlAgents")
    
    names = registered_agents.find_all(id=re.compile("ctl00_cphContent_rptAgents_ct(...)_lblAgentName"))
    names = [name.get_text() for name in names]
    city_state_zips = registered_agents.find_all(id=re.compile("ctl00_cphContent_rptAgents_ct(...)_lblAgentCSZ"))
    city_state_zips = [csz.get_text() for csz in city_state_zips]
    addresses = registered_agents.find_all(id=re.compile("ctl00_cphContent_rptAgents_ct(...)_lblAgentAddress1"))
    addresses = [addr.get_text() for addr in addresses]

    num_people = len(names)
    
    all_details = []
#     for i in range(num_people):
#         details["Full Name " + str(i + 1)] = names[i]
#         details["Address " + str(i + 1)] = addresses[i]
#         details["City,State,Zip " + str(i + 1)] = city_state_zips[i]
    
    for i in range(num_people):
        details = {}
        details['Owner Name'] = business_name
        details["Full Name"] = names[i]
        details["Address"] = addresses[i]
        details["City,State,Zip"] = city_state_zips[i]
        all_details.append(details)

#     print(all_details)
    return all_details

In [7]:
def navigate_best_match(business_name,search_results_page):
    soup = BeautifulSoup(search_results_page, "lxml")
    search_results = soup.find(id="ctl00_cphContent_grdSearchResults_EntityNameOrCharterNumber")
    top_results = search_results.find_all(class_=["RowNormal","RowAlt"])[:5]
    top_results_names = [result.td.get_text() for result in top_results]
#     print('Top 5 Search Results are')
#     print(top_results_names)
#     print('\n')
    match_index = None
    for i,name in enumerate(top_results_names):
        if business_name[:5] in name: # checks if first 5 chars in match business name
            match_index = i
            break

    details_button_id = top_results[match_index].input.attrs['id']
    driver.find_element_by_id(details_button_id).click()

In [8]:
def get_all_business_data(businesses):
    business_data = []
    for business in businesses:
        print('getting details for: ' + business)
        search(business)
        page = driver.page_source
        if not is_on_business_page(page):
            navigate_best_match(business,page)

        business_page = driver.page_source
        business_details = get_business_details(business,business_page)
        for detail in business_details:
            business_data.append(detail)
        
    return business_data
        

In [9]:
business_data = get_all_business_data(TEST_BUSINESSES)

getting details for: Whirlwind space ac llc
getting details for: T & T Real Estate Dev LLC
getting details for: R2h Investments LLC


In [10]:
business_data

[{'Address': '2312 NAPOLEON AVENUE',
  'City,State,Zip': 'NEW ORLEANS, LA  70115',
  'Full Name': 'ANDREW BERNARD',
  'Owner Name': 'Whirlwind space ac llc'},
 {'Address': '1001 VETERANS MEMORIAL BLVD., STE. 106',
  'City,State,Zip': 'KENNER, LA  70062',
  'Full Name': 'ALEX TAN',
  'Owner Name': 'T & T Real Estate Dev LLC'},
 {'Address': '1001 VETERANS MEMORIAL BLVD., STE. 106',
  'City,State,Zip': 'KENNER, LA  70062',
  'Full Name': 'LANCE THEARD',
  'Owner Name': 'T & T Real Estate Dev LLC'},
 {'Address': '313 OPELOUSAS AVE',
  'City,State,Zip': 'NEW ORLEANS, LA  70114',
  'Full Name': 'MICHAEL JOHNSTON',
  'Owner Name': 'R2h Investments LLC'}]

In [11]:
business_df = pd.DataFrame(business_data)

In [12]:
business_df.head()

,Address,"City,State,Zip",Full Name,Owner Name
0,2312 NAPOLEON AVENUE,"NEW ORLEANS, LA 70115",ANDREW BERNARD,Whirlwind space ac llc
1,"1001 VETERANS MEMORIAL BLVD., STE. 106","KENNER, LA 70062",ALEX TAN,T & T Real Estate Dev LLC
2,"1001 VETERANS MEMORIAL BLVD., STE. 106","KENNER, LA 70062",LANCE THEARD,T & T Real Estate Dev LLC
3,313 OPELOUSAS AVE,"NEW ORLEANS, LA 70114",MICHAEL JOHNSTON,R2h Investments LLC


In [13]:
business_df['Mailing Zip Code'] = business_df['City,State,Zip'].apply(lambda x: x.split()[-1])
business_df['Mailing State'] = business_df['City,State,Zip'].apply(lambda x: x.split()[-2])
business_df['Mailing City'] = business_df['City,State,Zip'].apply(lambda x: x.split(',')[0])

business_df['First Name'] = business_df['Full Name'].apply(lambda x: x.split()[0])
business_df['Last Name'] = business_df['Full Name'].apply(lambda x: x.split()[1])
business_df.head()

,Address,"City,State,Zip",Full Name,Owner Name,Mailing Zip Code,Mailing State,Mailing City,First Name,Last Name
0,2312 NAPOLEON AVENUE,"NEW ORLEANS, LA 70115",ANDREW BERNARD,Whirlwind space ac llc,70115,LA,NEW ORLEANS,ANDREW,BERNARD
1,"1001 VETERANS MEMORIAL BLVD., STE. 106","KENNER, LA 70062",ALEX TAN,T & T Real Estate Dev LLC,70062,LA,KENNER,ALEX,TAN
2,"1001 VETERANS MEMORIAL BLVD., STE. 106","KENNER, LA 70062",LANCE THEARD,T & T Real Estate Dev LLC,70062,LA,KENNER,LANCE,THEARD
3,313 OPELOUSAS AVE,"NEW ORLEANS, LA 70114",MICHAEL JOHNSTON,R2h Investments LLC,70114,LA,NEW ORLEANS,MICHAEL,JOHNSTON


In [14]:
business_df = business_df[['Owner Name', 'First Name', 'Last Name', 'Full Name',
        'Address','Mailing City', 'Mailing State','Mailing Zip Code','City,State,Zip']]
business_df.head()

,Owner Name,First Name,Last Name,Full Name,Address,Mailing City,Mailing State,Mailing Zip Code,"City,State,Zip"
0,Whirlwind space ac llc,ANDREW,BERNARD,ANDREW BERNARD,2312 NAPOLEON AVENUE,NEW ORLEANS,LA,70115,"NEW ORLEANS, LA 70115"
1,T & T Real Estate Dev LLC,ALEX,TAN,ALEX TAN,"1001 VETERANS MEMORIAL BLVD., STE. 106",KENNER,LA,70062,"KENNER, LA 70062"
2,T & T Real Estate Dev LLC,LANCE,THEARD,LANCE THEARD,"1001 VETERANS MEMORIAL BLVD., STE. 106",KENNER,LA,70062,"KENNER, LA 70062"
3,R2h Investments LLC,MICHAEL,JOHNSTON,MICHAEL JOHNSTON,313 OPELOUSAS AVE,NEW ORLEANS,LA,70114,"NEW ORLEANS, LA 70114"


In [15]:
# DF TO EXCEL

writer = pd.ExcelWriter('louisiana.xlsx')
business_df.to_excel(writer,'Sheet1')
writer.save()

# DF TO CSV
business_df.to_csv('louisiana.csv', sep=',')

In [16]:
service.stop()